<a href="https://colab.research.google.com/github/johyunkang/MLwithPythonCookbook/blob/main/13_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 13.1 직선 학습하기

과제 : 특성과 타깃 벡터 사이의 선형 관계를 표현하는 모델을 훈련하고 싶음

해결 : 선형 회귀를 사용함(`LinearRegression`)

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
import warnings
warnings.filterwarnings(action='ignore')

boston = load_boston()
x = boston.data[:, :2] # 특성 2개만 선택
print('feature shape:', x.shape)
print('feature sample:', x[:3])
y = boston.target
print('target shape:', y.shape)
print('target sample:', y[:3])
print('feature name:', boston.feature_names)


lr = LinearRegression()
model = lr.fit(x, y)

feature shape: (506, 2)
feature sample: [[6.320e-03 1.800e+01]
 [2.731e-02 0.000e+00]
 [2.729e-02 0.000e+00]]
target shape: (506,)
target sample: [24.  21.6 34.7]
feature name: ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [2]:
# 절편(intercept) = 편향(bias)
print('Intercept(=bias) :', model.intercept_)
print('coef :', model.coef_)
# dir(model)

print('\n실제값:', y[0] * 1000) # 보스턴 주택가격 단위가 천 달러라, 1000을 곱해줌
print('예측값:', model.predict(x)[0] * 1000)

Intercept(=bias) : 22.485628113468223
coef : [-0.35207832  0.11610909]

실제값: 24000.0
예측값: 24573.366631705547


## 13.2 교차 특성 다루기

과제 : 타깃 변수에 영향을 미치면서 다른 특성에 의존하는 특성이 있음

해결 : 사이킷런의 `PolynomialFeatures` 클래스로 교차항(interactive term)을 만들어 의존성을 잡아냅니다.

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings(action='ignore')

boston = load_boston()
x = boston.data[:, :2]
y = boston.target
class_name = boston.feature_names
# dir(boston)
print('class name 2개:', class_name[:2])

# 교차항을 생성
# degree : 교차항을 만들 때 최대 특성의 수
# include_bias : 기본적으로 절편(bias)이라 부르는 1로 채워진 특성을 추가하는데, False 이면 그렇게 하지 않음
# interaction_only : True 지정 시 오직 교차항만 반환 
interaction = PolynomialFeatures(degree=3, include_bias=False, interaction_only=True)

features_interaction = interaction.fit_transform(x)
print('feature interaction sample:', features_interaction[:3])

lr = LinearRegression()
model = lr.fit(features_interaction, y)


class name 2개: ['CRIM' 'ZN']
feature interaction sample: [[6.3200e-03 1.8000e+01 1.1376e-01]
 [2.7310e-02 0.0000e+00 0.0000e+00]
 [2.7290e-02 0.0000e+00 0.0000e+00]]


In [4]:
print('첫 번째 샘플의 특성값 확인:', x[0])

import numpy as np
#각 샘플에서 첫 번째와 두 번째 특성을 곱한다.
interaction_term = np.multiply(x[:, 0], x[:, 1])

print('첫 번째 샘플의 교차항 확인:', interaction_term[0])
print('위의 "feature interaction sample" 값과 "첫번째 샘플의 특성값" 과 바로 위 "교차항 확인" 값이 동일한 것을 확인할 수 있다')

첫 번째 샘플의 특성값 확인: [6.32e-03 1.80e+01]
첫 번째 샘플의 교차항 확인: 0.11376
위의 "feature interaction sample" 값과 "첫번째 샘플의 특성값" 과 바로 위 "교차항 확인" 값이 동일한 것을 확인할 수 있다


## 13.4 규제로 분산 줄이기

과제 : 선형 회귀의 분산을 줄이고 싶습니다.

해결 : 리지 회귀나 라소 회귀와 같이 축소 페널티( 또는 규제 regularization)가 포함된 학습 알고리즘을 사용

In [5]:
from sklearn.linear_model import Ridge
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

boston = load_boston()
x = boston.data
y = boston.target
class_name = boston.feature_names
print('class_name sample:', class_name[:2])

# 표준화
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# alpha 값을 지정한 리지 회귀 생성
ridge = Ridge(alpha=0.5)

model = ridge.fit(x_scaled, y)


class_name sample: ['CRIM' 'ZN']


표준 선형 회귀 모델에서는 정답(y)와 예측(y_hat) 사이의 제곱 오차 합 (Sum of Squared Error) 또는 잔차 제곱 합(Residual Sum of Squared)을 최소화 하기 위해 훈련함

$RSS = \displaystyle \sum_{i=1}^n(y_i - \hat{y}_i)^2$


- Ridge 회귀 : 축소 페널티는 모든 계수의 제곱합에 튜닝 파라미터를 곱한 것

    $RSS + \alpha \displaystyle \sum_{j=1}^p \hat{\beta}_j^2$

    - b_hat : p 특성의 j번째 계수
    - a : 하이퍼파라미터

- Lasso 회귀 : 축소 페널티가 모든 계수의 절댓값 합에 튜닝 하이퍼파라미터를 곱한 것

    $\dfrac {1} {2n} RSS + \alpha \displaystyle \sum_{j=1}^p \left|\hat{\beta}_j \right|$

    - n : 샘플의 갯수


일반적으로 라소가 더 이해하기 쉬운 모델을 만듦. 대신 리지 회귀가 라소 보다 더 좋은 예측을 만듦. 리지와 라소 사이 균형을 맞추고 싶으면 **엘라스틱 넷**을 사용하면 됨

- 하이퍼파라미터 a (alpha) : 계수를 얼마나 불리하게 만들지 조절. a 값이 클수록 더 간단한 모델을 만듦. 이상적인 a를 구하려면 `RidgeCV` 클래스를 사용


In [8]:
from sklearn.linear_model import RidgeCV

rcv = RidgeCV(alphas=[0.1, 1.0, 10.0])
model_cv = rcv.fit(x_scaled, y)

print('coef :', model_cv.coef_)

print('\nalpha value:', model_cv.alpha_)

coef : [-0.91987132  1.06646104  0.11738487  0.68512693 -2.02901013  2.68275376
  0.01315848 -3.07733968  2.59153764 -2.0105579  -2.05238455  0.84884839
 -3.73066646]

alpha value: 1.0


## 13.5 라소 회귀로 특성 줄이기

과제 : 특성의 수를 줄여서 선영 회귀 모델을 단순하게 만들고 싶음

해결 : 라소 회귀를 사용합니다.

In [17]:
from sklearn.linear_model import Lasso
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# set_printoptions : 지수를 실수로 표현
# suppress : True 면 무조건 실수, False면 엄청 작은 수는 그대로 지수로 표현
np.set_printoptions(precision=5, suppress=False) 

# 지수 표현 없애기
# pd.options.display.float_format='{:.5f}'.format

# 지수표현 원복
# pd.reset_option('display.float_format')

boston = load_boston()
x = boston.data
y = boston.target

print('x sample:', x[:3])

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
print('\nx_scaled sample:', x_scaled[:3])

# a 값을 지정한 라쏘 회귀
lasso = Lasso(alpha=0.5)

model = lasso.fit(x_scaled, y)

print('\n\n모델 계수:', model.coef_)

x sample: [[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 7.1850e+00
  6.1100e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9283e+02
  4.0300e+00]]

x_scaled sample: [[-0.41978  0.28483 -1.28791 -0.2726  -0.14422  0.41367 -0.12001  0.14021
  -0.98284 -0.66661 -1.459    0.44105 -1.07556]
 [-0.41734 -0.48772 -0.59338 -0.2726  -0.74026  0.19427  0.36717  0.55716
  -0.86788 -0.98733 -0.30309  0.44105 -0.49244]
 [-0.41734 -0.48772 -0.59338 -0.2726  -0.74026  1.28271 -0.26581  0.55716
  -0.86788 -0.98733 -0.30309  0.39643 -1.20873]]


모델 계수: [-0.11526  0.      -0.       0.39708 -0.       2.97426 -0.      -0.17057
 -0.      -0.      -1.59845  0.54314 -3.66614]


라소 회귀 페널티 특성은 모델의 계수를 0까지 축소시킬 수 있다는 것

위에서 `alpha`를 0.5로 지정하여 많은 계수가 0이 되었음. 즉 0에 해당하는 특성은 모델에서 사용되지 않는 다는 의미

In [18]:
# a 값이 너무 크게 증가하면 어떤 특성도 사용되지 않음
lasso_a10 = Lasso(alpha=10)

model_a10 = lasso_a10.fit(x_scaled, y)
print('a 값이 증가한 모델 계수:', model_a10.coef_)
print('\n a 값이 너무 크면 어떠한 특성도 사용되지 않음')

a 값이 증가한 모델 계수: [-0.  0. -0.  0. -0.  0. -0.  0. -0. -0. -0.  0. -0.]

 a 값이 너무 크면 어떠한 특성도 사용되지 않음


라소의 a 값을 쉽게 찾기 위해 `LassoCV` 클래스를 이용 가능

`cv` 옵션을 통해 n 폴드 교차검증 사용할 수 있음

In [20]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(alphas=[0.1, 1.0, 10], cv=5)

model_cv = lasso_cv.fit(x_scaled, y)

print('계수:', model_cv.coef_)

print('\n 최고의 a값:', model_cv.alpha_)

계수: [-0.6323   0.70841 -0.       0.65761 -1.57419  2.82627 -0.      -2.42208
  1.19594 -0.84647 -1.92249  0.76217 -3.72618]

 최고의 a값: 0.1


`LassoCV`는 탐색할 a 값을 명시적으로 지정하지 않고, `n_alphas` 를 통해 자동으로 탐색 대상 값을 생성할 수 있음

`n_alphas`의 디폴트 값은 100이며, 이를 1,000 개로 늘려 테스트 해보자.

In [21]:
lasso_cv_1000 = LassoCV(n_alphas=1000, cv=5)

model_cv_1000 = lasso_cv_1000.fit(x_scaled, y)

print('1000개의 후보 중 선정된 a :', model_cv_1000.alpha_)

1000개의 후보 중 선정된 a : 0.15326173083090813
